In [1]:
import numpy as np
import random

In [10]:
#sample data, change later

num_universes = 50
num_variables = 6
max_iterations = 1000
P_demand = 2000

lower_bound = np.zeros(num_variables)
upper_bound = np.ones(num_variables) * 1000

alpha = np.array([0.0027, 0.0028, 0.0026, 0.0055, 0.0055, 0.0083])
beta = np.array([17.83, 17.54, 17.23, 29.30, 29.58, 75.73])
gamma = np.array([4.46, 4.45, 4.44, 4.45, 4.46, 5.21])


In [11]:
def fitness_function_with_penalty(universe, P_demand, penalty_factor=10000):
    P_total = np.sum(universe)

    original_cost = np.sum(alpha * universe**2 + beta * universe + gamma)

    penalty = penalty_factor * max(0, P_demand - P_total)

    penalized_cost = original_cost + penalty

    return penalized_cost


In [4]:
def initialize_universes(num_universes, num_variables, lower_bound, upper_bound):
    return np.random.uniform(low=lower_bound, high=upper_bound, size=(num_universes, num_variables))

universes = initialize_universes(num_universes, num_variables, lower_bound, upper_bound)


In [12]:
fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])


In [6]:
def white_black_hole_mechanism(universes, fitness):
    sorted_indices = np.argsort(fitness)
    best_universe = universes[sorted_indices[0]]
    worst_universe = universes[sorted_indices[-1]]

    for i in range(num_universes):
        if i != sorted_indices[0]:
            universes[i] += np.random.uniform(0, 1) * (best_universe - universes[i])
            universes[i] -= np.random.uniform(0, 1) * (universes[i] - worst_universe)

    return universes


In [7]:
def wormhole_mechanism(universes, iteration, max_iterations, lower_bound, upper_bound):
    WEP = 0.2 + iteration * (1.0 - 0.2) / max_iterations
    TDR = 1 - (iteration / max_iterations)**6

    for i in range(num_universes):
        if np.random.uniform(0, 1) < WEP:
            universes[i] += TDR * np.random.uniform(-1, 1, size=num_variables) * (upper_bound - lower_bound)
            universes[i] = np.clip(universes[i], lower_bound, upper_bound)

    return universes


In [13]:
best_solutions = []

for iteration in range(max_iterations):
    universes = white_black_hole_mechanism(universes, fitness)

    universes = wormhole_mechanism(universes, iteration, max_iterations, lower_bound, upper_bound)

    fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])

    best_fitness_idx = np.argmin(fitness)
    best_solutions.append(universes[best_fitness_idx])


In [15]:
best_universe = min(best_solutions, key=lambda u: fitness_function_with_penalty(u, P_demand))
min_cost = fitness_function_with_penalty(best_universe, P_demand)

print("Best solution (power outputs):", best_universe)
print("Minimum generation cost:", min_cost)


Best solution (power outputs): [677.74806917 474.98769494 343.94789113  11.93552119 499.11750259
   3.58511709]
Minimum generation cost: 45304.89482738727
